Solve the following qubo problem for maximum value. 

$ y = x_1+ x_2 + x_3 + x_4 $ with the constraints below. 

- $ x_1 + x_3 + x_4 <= 1 $
- $ x_1 + x_2 <= 1 $


In [34]:
from qiskit_optimization import QuadraticProgram
from qiskit import BasicAer
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.converters import MinimizeToMaximize, QuadraticProgramToQubo

Construct the qubo matrix that satisfies the equation below and with constaints above.

$ X^T Q X $

In [51]:
Q = QuadraticProgram("qproblem")
[Q.binary_var() for _ in range(4)]
Q.linear_constraint(linear={"x0": 1, "x2": 1, "x3": 1}, sense="<=", rhs=1, name="lin_eq1")
Q.linear_constraint(linear={"x0": 1, "x1": 1}, sense="<=", rhs=1, name="lin_eq2")
Q.maximize(linear=[1,1,1,1], quadratic={("x1", "x3"):-50, ("x0", "x3"): -50, ("x2", "x3"): -50, ("x0", "x1"):-50})
print(Q.prettyprint())

Problem name: qproblem

Maximize
  -50*x0*x1 - 50*x0*x3 - 50*x1*x3 - 50*x2*x3 + x0 + x1 + x2 + x3

Subject to
  Linear constraints (2)
    x0 + x2 + x3 <= 1  'lin_eq1'
    x0 + x1 <= 1  'lin_eq2'

  Binary variables (4)
    x0 x1 x2 x3



In [52]:
quantum_instance = Sampler()
qaoa_mes = QAOA(quantum_instance, COBYLA(), reps=2)
qaoa = MinimumEigenOptimizer(qaoa_mes)

qaoa_result = qaoa.solve(Q)
[qaoa_result.x], [qaoa_result.fval]


([array([0., 1., 1., 0.])], [2.0])